<a href="https://colab.research.google.com/github/seoho0529/TensorFlow/blob/main/tf24count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# 자연어 처리에서 특징추출 이란 단어나 문장들을 어떤 특징 값으로 변환하는 것을 의미한다.
# 문자로 구성된 데이터를 모델에 적용할 수 있도록 특징을 추출해 수치화한다.
# 단어의 수를 파악해 문장을 분석하는 방법

from sklearn.feature_extraction.text import CountVectorizer

textData = ['나는 배 고프다 아니 배가 고프다.', '내일 점심 뭐 먹지?', '내일 공부 해야겠다.', '점심먹고 공부 해야지!']
count_vec = CountVectorizer(analyzer='word', ngram_range=(1,1), stop_words=['나는'])  # ngram_range:단어장 생성에 사용할 토큰 크기를 결정
count_vec.fit(textData)
print(count_vec.get_feature_names_out())  # konlpy를 사용하면 조사,형용사 등 뺄 수 있음
print(count_vec.vocabulary_)  # 사전 순으로 인덱싱
print([textData[0]])
sentence = [textData[0]]
print(count_vec.transform(sentence))  # 0행에서 '고프다' 2번, 0행에서 '배' 1번, 0행에서 '아니' 1번  | 벡터화
print(count_vec.transform(sentence).toarray())

['고프다' '공부' '내일' '먹지' '배가' '아니' '점심' '점심먹고' '해야겠다' '해야지']
{'고프다': 0, '아니': 5, '배가': 4, '내일': 2, '점심': 6, '먹지': 3, '공부': 1, '해야겠다': 8, '점심먹고': 7, '해야지': 9}
['나는 배 고프다 아니 배가 고프다.']
  (0, 0)	2
  (0, 4)	1
  (0, 5)	1
[[2 0 0 0 1 1 0 0 0 0]]


In [13]:
# TF-IDF(Term Frequency(1개의 문장 내 특정 단어 등장빈도) - Inverse Document Frequency(DF:특정 단어가 나타나는 문장 수))
# 정보 검색과 텍스트 마이닝에서 이용하는 가중치로, 여러 문서로 이루어진 문서군이 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치이다.
# 단순히 빈도수로 그 단어의 가치를 정하는 것이 아니라, 여러 문장에 많이 등장하는 단어는 패널티를 주어 단어 빈도의 scale을 맞추는 기법
# 즉, 분별력 있는 특징을 만들어준다.


from sklearn.feature_extraction.text import TfidfVectorizer

textData = ['나는 배 고프다 아니 배가 고프다.', '내일 점심 뭐 먹지?', '내일 공부 해야겠다.', '점심먹고 공부 해야지!']
tfidf_vec = TfidfVectorizer(analyzer='word', ngram_range=(1,1), stop_words=['나는'])
tfidf_vec.fit(textData)
print(tfidf_vec.get_feature_names_out())
print(tfidf_vec.vocabulary_)
print(tfidf_vec.transform(textData).toarray())  # 확률값으로 나옴
print()
sentence = [textData[3]]
print(sentence)
print(tfidf_vec.transform(sentence))
print(tfidf_vec.transform(sentence).toarray()) # 먹고,해야지가 점심 공부 보다 중요하다고 판단

['고프다' '공부' '내일' '먹지' '배가' '아니' '점심' '점심먹고' '해야겠다' '해야지']
{'고프다': 0, '아니': 5, '배가': 4, '내일': 2, '점심': 6, '먹지': 3, '공부': 1, '해야겠다': 8, '점심먹고': 7, '해야지': 9}
[[0.81649658 0.         0.         0.         0.40824829 0.40824829
  0.         0.         0.         0.        ]
 [0.         0.         0.48693426 0.61761437 0.         0.
  0.61761437 0.         0.         0.        ]
 [0.         0.52640543 0.52640543 0.         0.         0.
  0.         0.         0.66767854 0.        ]
 [0.         0.48693426 0.         0.         0.         0.
  0.         0.61761437 0.         0.61761437]]

['점심먹고 공부 해야지!']
  (0, 9)	0.6176143709756019
  (0, 7)	0.6176143709756019
  (0, 1)	0.48693426407352264
[[0.         0.48693426 0.         0.         0.         0.
  0.         0.61761437 0.         0.61761437]]


In [15]:
# TfidVectorizer를 사용해 텍스트(형태소 분석기 Okt를 적용)를 벡터로 변환 후 단어 간 유사도 계산
# pip install konlpy
from konlpy.tag import Okt
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
okt = Okt()

def tokenizeFunc(ss):   # 형태소 분석용 함수
  ss = okt.normalize(ss) # Okt는 정규화를 일부 지원함. ex)'사랑햌' -> '사랑해'로 수정
  ss = okt.morphs(ss)    # 형태소 단위로 분리, 반환형은 리스트
  return ss

texts = ['길동이는 파이썬을 좋아합니다','길동이는 웹을 잘합니다','길동이는 운동을 매우 잘합니다']
new_texts = ['길동이는 파이썬을 좋아하고 운동을 잘합니다']

tfidf = TfidfVectorizer(tokenizer=tokenizeFunc, token_pattern=None).fit(texts)
tfidf_matrix = tfidf.fit_transform(texts)
print(tfidf_matrix)
print(tfidf_matrix.toarray())

  (0, 8)	0.5427006131762078
  (0, 5)	0.32052772458725637
  (0, 9)	0.5427006131762078
  (0, 1)	0.32052772458725637
  (0, 6)	0.32052772458725637
  (0, 0)	0.32052772458725637
  (1, 10)	0.40352535506797127
  (1, 7)	0.40352535506797127
  (1, 4)	0.5305873490316616
  (1, 5)	0.31337343564910264
  (1, 1)	0.31337343564910264
  (1, 6)	0.31337343564910264
  (1, 0)	0.31337343564910264
  (2, 2)	0.4686986463592043
  (2, 3)	0.4686986463592043
  (2, 10)	0.356457401476207
  (2, 7)	0.356457401476207
  (2, 5)	0.27682097087637686
  (2, 1)	0.27682097087637686
  (2, 6)	0.27682097087637686
  (2, 0)	0.27682097087637686
[[0.32052772 0.32052772 0.         0.         0.         0.32052772
  0.32052772 0.         0.54270061 0.54270061 0.        ]
 [0.31337344 0.31337344 0.         0.         0.53058735 0.31337344
  0.31337344 0.40352536 0.         0.         0.40352536]
 [0.27682097 0.27682097 0.46869865 0.46869865 0.         0.27682097
  0.27682097 0.3564574  0.         0.         0.3564574 ]]


In [23]:
for ntext in new_texts:
  tftrans = tfidf.transform([ntext])  # 새로운 문장을 벡터로 변환
  print('tftrans : \n', tftrans)
  # 새로운 문장과 기존 문장들 사이의 코사인 유사도 계산(데이터 크기 차이에 관계없이 계산)
  cosine_simil = cosine_similarity(tftrans, tfidf_matrix)
  print('cosine_simil : ', cosine_simil)  # [[0.6294     0.65051252 0.77272178]]

  # 출력
  print(f'새로운 문장 : {ntext}')
  print('------------')
  print(f'기존 문장 : ')
  for idx in range(3):  # 기존문장이 3개니까 3개
    # print(cosine_simil.argsort()[0])  # 새로운 문장과 기존 문장 3개를 비교, [0]은 2차원 배열의 0행을 의미
    # print((idx + 1) * -1)
    print(cosine_simil[0][(idx + 1) * -1])
    most_simil_idx = cosine_simil.argsort()[0][(idx + 1) * -1]
    print(most_simil_idx)
    most_simil_sentence = texts[most_simil_idx]
    simil_score = cosine_simil[0][most_simil_idx]
    print(f'{most_simil_sentence} (유사도:{simil_score:.3f})')
  print()

  # 순서가 있는 자연어 데이터를 통해 문장생성, 챗봇, 문서 요약, 이미지 설명, 주식예측, 문장을 소리로 변역 ...을 하려면 RNN을 사

tftrans : 
   (0, 10)	0.3214212466114349
  (0, 9)	0.4226303131144919
  (0, 7)	0.3214212466114349
  (0, 6)	0.2496122711403758
  (0, 5)	0.4992245422807516
  (0, 3)	0.4226303131144919
  (0, 1)	0.2496122711403758
  (0, 0)	0.2496122711403758
cosine_simil :  [[0.6294     0.65051252 0.77272178]]
새로운 문장 : 길동이는 파이썬을 좋아하고 운동을 잘합니다
------------
기존 문장 : 
0.7727217765585208
2
길동이는 운동을 매우 잘합니다 (유사도:0.773)
0.6505125202677131
1
길동이는 웹을 잘합니다 (유사도:0.651)
0.6293999965624972
0
길동이는 파이썬을 좋아합니다 (유사도:0.629)

